Step 1: scrape the Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M.

In [1]:
import urllib.request

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#url
page = urllib.request.urlopen(url) #import the url and get HTML to page

In [3]:
# import the BeautifulSoup library so we can parse HTML and XML documents
!pip install bs4
from bs4 import BeautifulSoup

     |████████████████████████████████| 122kB 6.9MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
Successfully built bs4


In [4]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "html.parser")

In [5]:
nbr_table=soup.find('table', class_='wikitable sortable')
#nbr_table

In [35]:
#There are 3 columns of interest - PostalCode, Borough, Neighborhood
PC_list=[]
B_list=[]
N_list=[]
for row in nbr_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        PC_list.append(cells[0].find(text=True).strip())
        B_list.append(cells[1].find(text=True).strip())
        N_list.append(cells[2].find(text=True).strip())

# check
# for x in range(5):
#   print (PC_list[x] + " " + B_list[x] + " " + N_list[x])


0

In [41]:
import pandas as pd
df_nbr_raw = pd.DataFrame(PC_list, columns=['PostalCode'])
df_nbr_raw['Borough'] = B_list
df_nbr_raw['Neighborhood'] = N_list
df_nbr_raw.head()
print ("Number of not assigned neighborhood = " + str(df_nbr_raw[df_nbr_raw.Neighborhood =='Not assigned'].shape[0]))
#df_nbr_raw.shape

Number of not assigned neighborhood = 77


In [42]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
row_ids = df_nbr_raw[df_nbr_raw.Borough.str.lower() == 'not assigned'].index
#index_names
df_nbr_raw.drop(row_ids, inplace=True)
df_nbr_raw.reset_index(drop=True, inplace=True)
df_nbr_raw.head()
#df_nbr_raw[df_nbr_raw['Neighborhood'] == 'Not assigned'] #shows that there are no neighborhoods that are not assigned
#df_nbr_raw.shape
#df_nbr_raw.info
print ("Number of not assigned neighborhood = " + str(df_nbr_raw[df_nbr_raw.Neighborhood =='Not assigned'].shape[0]))


Number of not assigned neighborhood = 0


In [26]:
#Combine two rows with different neighborhood into one row with the neighborhoods separated with a comma
df_tmp = df_nbr_raw.groupby(['PostalCode', 'Borough'], sort=False)['Neighborhood'].agg(','.join)
df_nbr = df_tmp.reset_index()
df_nbr.head(20)

pandas.core.frame.DataFrame

In [33]:
#If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
df_nbr.loc[(df_nbr.Neighborhood ==  'Not assigned'), 'Neighborhood'] = df_nbr.Borough
#df_nbr[df_nbr.Neighborhood =='Not assigned'].shape[0]

0

In [43]:
df_nbr.shape

(103, 3)